In [9]:
#import tensorflow and force eager execution for gradcam (numpy etc)

import tensorflow as tf
print("TensorFlow version:", tf.__version__) 

TensorFlow version: 1.13.1


In [10]:
use_device='/gpu:0'


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Input, Concatenate
from tensorflow.keras.layers import LSTM

from tensorflow.python.client import device_lib

#import models and layers from tensorflow.keras to avoid errors like "TypeError: Checkpointable._track_checkpointable()"
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dropout, Activation, Flatten
from keras.optimizers import Adam
    
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#import tensorflow.keras.backend as K
from keras import regularizers
    
import pickle
import pandas as pd

from datetime import datetime

from sklearn.model_selection import train_test_split
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4903214409589439056
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3072013108
locality {
  bus_id: 1
  links {
  }
}
incarnation: 765633940849601160
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [11]:
# define paths for saving results and other data
#---------------------------------------------------------------------------------------------------------------------    
path_data= "xxx" #where is the dataset
path_data_2= "xxx"


X1_import=path_data + "SMILES_1.txt" #where is the Xdata of the dataset (SMILES)
X2_import=path_data + "SMILES_2.txt"
# Load the saved numeric arrays from the file
Y_import=path_data + "K_GT.txt" #where is the Ydata of the dataset (property to predict)
   

# Load SMILES strings from file
with open(X1_import, 'r') as f:
    smiles_data_1 = f.readlines()
with open(X2_import, 'r') as f:
    smiles_data_2 = f.readlines()

# Load properties from file
with open(Y_import, 'r') as f:
    X_data = f.readlines()
    
print(smiles_data_1[10])
print(smiles_data_2[10])
print(X_data[10])

OC[C@H]1O[C@H](O[C@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)[C@H](O)[C@@H](O)[C@@H]1O

OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C@@H]2O)[C@H](O)[C@@H](O)[C@@H]1O

0.56



In [12]:
# Split the dataset into train and test sets
# Load the tokenizer
tokenizer_path = path_data_2 + 'tokenizer_all.pkl'
with open(tokenizer_path, 'rb') as f:
    tokenizer_all = pickle.load(f)

# Preprocess SMILES data for SMILES_1
sequences_1 = tokenizer_all.texts_to_sequences(smiles_data_1)
max_length_1 = max(len(seq) for seq in sequences_1)
padded_sequences_1 = pad_sequences(sequences_1, maxlen=max_length_1, padding='post')
print(np.shape(padded_sequences_1))

# Preprocess SMILES data for SMILES_2
sequences_2 = tokenizer_all.texts_to_sequences(smiles_data_2)
max_length_2 = max(len(seq) for seq in sequences_2)
padded_sequences_2 = pad_sequences(sequences_2, maxlen=max_length_2, padding='post')
print(np.shape(padded_sequences_2))

# Convert property data to numpy array
X_labels = np.array([float(prop.strip()) for prop in X_data])

# Define vocabulary size for SMILES_1
vocab_size_1 = len(tokenizer_all.word_index) + 1

# Define vocabulary size for SMILES_2
vocab_size_2 = len(tokenizer_all.word_index) + 1

# Define embedding dimension
embedding_dim = 128

# Split the dataset into train and test sets
test_size = 0.5
random_state = 42
X_train_1, X_test_1, X_train_2, X_test_2, y_train, y_test = train_test_split(
    padded_sequences_1, padded_sequences_2, X_labels, test_size=test_size, random_state=random_state)
print(f"X_train_1 shape: {X_train_1.shape}, X_train_2 shape: {X_train_2.shape}")
print(np.shape(y_train))


print(f"X_test_1 shape: {X_test_1.shape}, X_test_2 shape: {X_test_2.shape}")
print(np.shape(y_test))

(40, 237)
(40, 70)
X_train_1 shape: (20, 237), X_train_2 shape: (20, 70)
(20,)
X_test_1 shape: (20, 237), X_test_2 shape: (20, 70)
(20,)


In [13]:
#save X_test_1 and y_test to .npy file

np.save(path_data_2 + 'X_test_0.npy', X_test_1)
np.save(path_data_2 + 'X_test_1.npy', X_test_2)
np.save(path_data_2 + 'y_test.npy', y_test)

np.save(path_data_2 + 'X_train_0.npy', X_train_1)
np.save(path_data_2 + 'X_train_1.npy', X_train_2)
np.save(path_data_2 + 'y_train.npy', y_train)

In [14]:
from keras.callbacks import Callback
    
# checkpoint_filepath = path_data_2 + "Weights.npy" #'checkpoint'
monitor_custom='val_mean_absolute_error' #val_mean_absolute_error   val_mean_absolute_percentage_error
mode_custom='min'

early_stop= tf.keras.callbacks.EarlyStopping(monitor=monitor_custom,  verbose=3,
                                             mode=mode_custom, patience=250, restore_best_weights=True)

callback_list= [early_stop]




def Model_embeddings(X_train_1, X_train_2, y_train, vocab_size_1, vocab_size_2, embedding_dim, max_length_1, max_length_2,
                     ep, b_s, v_s, shuffle_dataset, alpha, NeuronsFC, w1, w2, w3, channels_1, channels_2, 
                     channels_3, lr, beta_1, beta_2, epsilon, decay, amsgrad):

    
    # Create input shapes
    input_shape1 = X_train_1.shape[1:]
    input_shape2 = X_train_2.shape[1:]
    # Define inputs
    A_input = Input(shape=input_shape1, name='INPUT')
    A2_input = Input(shape=input_shape2, name='INPUT_2')
    # ===================== Comment all experts code and get reference =================

    # Define first input network
#     A = Embedding(input_dim=vocab_size_1, output_dim=embedding_dim, input_length=max_length_1)(A_input)
#     A = Conv1D(channels_1, w1)(A)
#     A = tf.keras.layers.LeakyReLU(alpha)(A)
#     A = Conv1D(channels_2, w2)(A)
#     A = tf.keras.layers.LeakyReLU(alpha)(A)
#     A = Conv1D(channels_3, w3)(A)
#     A = tf.keras.layers.LeakyReLU(alpha)(A)
    A = Flatten()(A_input)#(A)
#     A = GlobalMaxPooling1D()(A)
    # Define second input network
#     A2 = Embedding(input_dim=vocab_size_2, output_dim=embedding_dim, input_length=max_length_2)(A2_input)
#     A2 = Conv1D(channels_1, w1)(A2)
#     A2 = tf.keras.layers.LeakyReLU(alpha)(A2)
#     A2 = Conv1D(channels_2, w2)(A2)
#     A2 = tf.keras.layers.LeakyReLU(alpha)(A2)
#     A2 = Conv1D(channels_3, w3)(A2)
#     A2 = tf.keras.layers.LeakyReLU(alpha)(A2)
    A2 = Flatten()(A2_input)#(A2)
#     A2 = GlobalMaxPooling1D()(A2)
    # Concatenate the outputs of the two networks
    concatenated = tf.keras.layers.concatenate([A, A2])
    merged = Dense(NeuronsFC)(concatenated)
    merged = tf.keras.layers.LeakyReLU(alpha)(merged)
    output_layer = Dense(1, activation='linear')(merged)
    # Create model
    model = Model(inputs=[A_input, A2_input], outputs=output_layer)
    print(model.summary(line_length=None, positions=None, print_fn=None))
    # Compile model
    opt = tf.keras.optimizers.Adam(lr, beta_1, beta_2, epsilon, decay, amsgrad)
    model.compile(loss='mean_absolute_error', optimizer=opt,
                           metrics=['mean_absolute_error', 'mean_absolute_percentage_error'])
    Data_X=[X_train_1, X_train_2]
    # Train the model
    HISTORY = model.fit(Data_X, y_train, epochs=ep, batch_size=b_s, validation_split=v_s,
              shuffle=shuffle_dataset, callbacks=callback_list)
    
    return model, HISTORY

In [15]:
#DATA=====================================================================================================
Mode=0
#NUMBER OF TEST===========================================================================================
number_of_test = 1
#FREEZE_LAYERS============================================================================================
is_trainable_conv1_1=True #
is_trainable_conv1_2=True #
is_trainable_conv1_3=True #
is_trainable_conv1_4=True #
is_trainable_conv1_5=True
is_trainable_HIDDEN_A1_0=True #the CNN input side of the ANN
is_trainable_HIDDEN_A2_0=True #the Hbonds input side of the ANN (1:True, 2:True, 2:False)
is_trainable_HIDDEN_FC_all=True #concatenated FCs
#HYPERPARAMETERS==========================================================================================
lr=[0.01, 0.001, 0.0001]
beta_1=0.99
beta_2=0.999
epsilon=None
decay=0
amsgrad=True
b_s=[1000, 30, 15, 5, 2]
samples_on_control=0
samples_on_test=200
v_s= 0.5 #(samples_on_test+samples_on_control)/number_of_samples_X   
shuffle_dataset="False"
Drop_prob=0.3
alpha=0.05
reg=0.01
#WORKING PARAMETERS======================================================================================
number_epochs=5000 #
#FULLY CONNECTED=========================================================================================
NeuronsFC=50 # neurons combined FC
NeuronsFC_A2=5 #neurons in the Hbonds input
NeuronsFC_A1=5 #neurons in the CNN input
#CNN=====================================================================================================
w1=1 # must be the same if the model has been pre-trained
w2=6
w3=10
channels_1=32
channels_2=64
channels_3=128

In [16]:
# Iterate over each combination of learning rate and batch size

counter=0
Results=[]
for i in b_s:
    for j in lr:
        Name_saving="KGT_embedding_" + str(counter) + str(number_of_test)
        print("model name (to save): " + Name_saving)
        MODEL, HISTORY= Model_embeddings(X_train_1, X_train_2, y_train, vocab_size_1, vocab_size_2, embedding_dim, max_length_1, max_length_2,
                         number_epochs, i, v_s, shuffle_dataset, alpha, NeuronsFC, w1, w2, w3, channels_1, channels_2, channels_3,
                         j, beta_1, beta_2, epsilon, decay, amsgrad)
        #save model
        try:
            MODEL.save(path_data_2 + Name_saving + ".model")
            print("Model saved")
        except:
            print("Model saved?")
            try:
                MODEL.save(path_data_2 + Name_saving + ".model")
                print("Model now saved!")
            except Exception as e:
                print("Error saving model:", e)
            
        # Extract the mean absolute error from the training history
        mean_absolute_error_model = HISTORY.history['mean_absolute_error'][-1]  # Considering the last epoch error
        val_mean_absolute_error_model = HISTORY.history['val_mean_absolute_error'][-1]
        MAPE_model = HISTORY.history['mean_absolute_percentage_error'][-1]  # Considering the last epoch error
        val_MAPE_model = HISTORY.history['val_mean_absolute_percentage_error'][-1]
        # Append the results to the list
        Results.append([counter, j, i, mean_absolute_error_model, val_mean_absolute_error_model, MAPE_model, val_MAPE_model])            
        counter+=1

model name (to save): KGT_embedding_01
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
INPUT (InputLayer)              (None, 237)          0                                            
__________________________________________________________________________________________________
INPUT_2 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 237)          0           INPUT[0][0]                      
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 70)           0           INPUT_2[0][0]                    
______________________________________________________________________

Epoch 25/5000
10/10 [==============================] - 0s 1ms/sample - loss: 0.7299 - mean_absolute_error: 0.7299 - mean_absolute_percentage_error: 179.4720 - val_loss: 0.6475 - val_mean_absolute_error: 0.6475 - val_mean_absolute_percentage_error: 171.9082
Epoch 26/5000
10/10 [==============================] - 0s 1ms/sample - loss: 0.5126 - mean_absolute_error: 0.5126 - mean_absolute_percentage_error: 123.3562 - val_loss: 0.7250 - val_mean_absolute_error: 0.7250 - val_mean_absolute_percentage_error: 195.5422
Epoch 27/5000
10/10 [==============================] - 0s 1ms/sample - loss: 0.4112 - mean_absolute_error: 0.4112 - mean_absolute_percentage_error: 117.6322 - val_loss: 0.8377 - val_mean_absolute_error: 0.8377 - val_mean_absolute_percentage_error: 226.9019
Epoch 28/5000
10/10 [==============================] - 0s 1000us/sample - loss: 0.5024 - mean_absolute_error: 0.5024 - mean_absolute_percentage_error: 163.7149 - val_loss: 0.9140 - val_mean_absolute_error: 0.9140 - val_mean_absol

Epoch 57/5000
10/10 [==============================] - 0s 1ms/sample - loss: 0.6383 - mean_absolute_error: 0.6383 - mean_absolute_percentage_error: 175.0516 - val_loss: 0.6021 - val_mean_absolute_error: 0.6021 - val_mean_absolute_percentage_error: 173.2605
Epoch 58/5000
10/10 [==============================] - 0s 1ms/sample - loss: 0.5884 - mean_absolute_error: 0.5884 - mean_absolute_percentage_error: 157.4759 - val_loss: 0.5249 - val_mean_absolute_error: 0.5249 - val_mean_absolute_percentage_error: 146.9952
Epoch 59/5000
10/10 [==============================] - 0s 800us/sample - loss: 0.4771 - mean_absolute_error: 0.4771 - mean_absolute_percentage_error: 121.8338 - val_loss: 0.5106 - val_mean_absolute_error: 0.5106 - val_mean_absolute_percentage_error: 140.0903
Epoch 60/5000
10/10 [==============================] - 0s 1ms/sample - loss: 0.3661 - mean_absolute_error: 0.3661 - mean_absolute_percentage_error: 88.6597 - val_loss: 0.5439 - val_mean_absolute_error: 0.5439 - val_mean_absolut

Epoch 89/5000
10/10 [==============================] - 0s 2ms/sample - loss: 0.3284 - mean_absolute_error: 0.3284 - mean_absolute_percentage_error: 117.8407 - val_loss: 0.6006 - val_mean_absolute_error: 0.6006 - val_mean_absolute_percentage_error: 165.9815
Epoch 90/5000
10/10 [==============================] - 0s 2ms/sample - loss: 0.3573 - mean_absolute_error: 0.3573 - mean_absolute_percentage_error: 125.6310 - val_loss: 0.5912 - val_mean_absolute_error: 0.5912 - val_mean_absolute_percentage_error: 162.5407
Epoch 91/5000
10/10 [==============================] - 0s 3ms/sample - loss: 0.3467 - mean_absolute_error: 0.3467 - mean_absolute_percentage_error: 121.0813 - val_loss: 0.5480 - val_mean_absolute_error: 0.5480 - val_mean_absolute_percentage_error: 147.5565
Epoch 92/5000
10/10 [==============================] - 0s 2ms/sample - loss: 0.2980 - mean_absolute_error: 0.2980 - mean_absolute_percentage_error: 105.2931 - val_loss: 0.4727 - val_mean_absolute_error: 0.4727 - val_mean_absolute

Epoch 121/5000
10/10 [==============================] - 0s 2ms/sample - loss: 0.1363 - mean_absolute_error: 0.1363 - mean_absolute_percentage_error: 37.8127 - val_loss: 0.3332 - val_mean_absolute_error: 0.3332 - val_mean_absolute_percentage_error: 78.9012
Epoch 122/5000
10/10 [==============================] - 0s 2ms/sample - loss: 0.1115 - mean_absolute_error: 0.1115 - mean_absolute_percentage_error: 29.1102 - val_loss: 0.2976 - val_mean_absolute_error: 0.2976 - val_mean_absolute_percentage_error: 67.7337
Epoch 123/5000
10/10 [==============================] - 0s 2ms/sample - loss: 0.0718 - mean_absolute_error: 0.0718 - mean_absolute_percentage_error: 16.6352 - val_loss: 0.3349 - val_mean_absolute_error: 0.3349 - val_mean_absolute_percentage_error: 79.7410
Epoch 124/5000
10/10 [==============================] - 0s 2ms/sample - loss: 0.0805 - mean_absolute_error: 0.0805 - mean_absolute_percentage_error: 24.1165 - val_loss: 0.3871 - val_mean_absolute_error: 0.3871 - val_mean_absolute_pe

...

Epoch 193/5000
10/10 [==============================] - 0s 5ms/sample - loss: 0.0491 - mean_absolute_error: 0.0491 - mean_absolute_percentage_error: 16.0200 - val_loss: 1.8809 - val_mean_absolute_error: 1.8809 - val_mean_absolute_percentage_error: 508.4752
Epoch 194/5000
10/10 [==============================] - 0s 5ms/sample - loss: 0.0410 - mean_absolute_error: 0.0410 - mean_absolute_percentage_error: 12.7468 - val_loss: 1.8948 - val_mean_absolute_error: 1.8948 - val_mean_absolute_percentage_error: 513.0552
Epoch 195/5000
10/10 [==============================] - 0s 5ms/sample - loss: 0.0511 - mean_absolute_error: 0.0511 - mean_absolute_percentage_error: 17.6883 - val_loss: 1.8968 - val_mean_absolute_error: 1.8968 - val_mean_absolute_percentage_error: 513.8357
Epoch 196/5000
10/10 [==============================] - 0s 6ms/sample - loss: 0.0625 - mean_absolute_error: 0.0625 - mean_absolute_percentage_error: 20.6338 - val_loss: 1.9056 - val_mean_absolute_error: 1.9056 - val_mean_absolute

Epoch 225/5000
10/10 [==============================] - 0s 8ms/sample - loss: 0.0502 - mean_absolute_error: 0.0502 - mean_absolute_percentage_error: 14.4628 - val_loss: 1.9026 - val_mean_absolute_error: 1.9026 - val_mean_absolute_percentage_error: 515.2977
Epoch 226/5000
10/10 [==============================] - 0s 8ms/sample - loss: 0.0454 - mean_absolute_error: 0.0454 - mean_absolute_percentage_error: 11.8560 - val_loss: 1.9410 - val_mean_absolute_error: 1.9410 - val_mean_absolute_percentage_error: 526.7409
Epoch 227/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0974 - mean_absolute_error: 0.0974 - mean_absolute_percentage_error: 26.4791 - val_loss: 1.9526 - val_mean_absolute_error: 1.9526 - val_mean_absolute_percentage_error: 530.0376
Epoch 228/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.1091 - mean_absolute_error: 0.1091 - mean_absolute_percentage_error: 27.7934 - val_loss: 1.9371 - val_mean_absolute_error: 1.9371 - val_mean_absolute

Epoch 257/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0635 - mean_absolute_error: 0.0635 - mean_absolute_percentage_error: 18.2725 - val_loss: 1.8727 - val_mean_absolute_error: 1.8727 - val_mean_absolute_percentage_error: 507.6035
Epoch 258/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0734 - mean_absolute_error: 0.0734 - mean_absolute_percentage_error: 17.5879 - val_loss: 1.8410 - val_mean_absolute_error: 1.8410 - val_mean_absolute_percentage_error: 497.6762
Epoch 259/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0973 - mean_absolute_error: 0.0973 - mean_absolute_percentage_error: 24.5994 - val_loss: 1.8244 - val_mean_absolute_error: 1.8244 - val_mean_absolute_percentage_error: 492.2432
Epoch 260/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0974 - mean_absolute_error: 0.0974 - mean_absolute_percentage_error: 26.0135 - val_loss: 1.8419 - val_mean_absolute_error: 1.8419 - val_mean_absolute

Epoch 289/5000
10/10 [==============================] - 0s 10ms/sample - loss: 0.0305 - mean_absolute_error: 0.0305 - mean_absolute_percentage_error: 10.2499 - val_loss: 1.8831 - val_mean_absolute_error: 1.8831 - val_mean_absolute_percentage_error: 509.0011
Epoch 290/5000
10/10 [==============================] - 0s 10ms/sample - loss: 0.0397 - mean_absolute_error: 0.0397 - mean_absolute_percentage_error: 12.0934 - val_loss: 1.9003 - val_mean_absolute_error: 1.9003 - val_mean_absolute_percentage_error: 514.6915
Epoch 291/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0296 - mean_absolute_error: 0.0296 - mean_absolute_percentage_error: 8.9753 - val_loss: 1.8933 - val_mean_absolute_error: 1.8933 - val_mean_absolute_percentage_error: 513.0533
Epoch 292/5000
10/10 [==============================] - 0s 10ms/sample - loss: 0.0277 - mean_absolute_error: 0.0277 - mean_absolute_percentage_error: 7.4343 - val_loss: 1.8722 - val_mean_absolute_error: 1.8722 - val_mean_absolut

Epoch 321/5000
10/10 [==============================] - 0s 10ms/sample - loss: 0.0516 - mean_absolute_error: 0.0516 - mean_absolute_percentage_error: 16.6268 - val_loss: 1.9139 - val_mean_absolute_error: 1.9139 - val_mean_absolute_percentage_error: 519.1762
Epoch 322/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0367 - mean_absolute_error: 0.0367 - mean_absolute_percentage_error: 10.2614 - val_loss: 1.8917 - val_mean_absolute_error: 1.8917 - val_mean_absolute_percentage_error: 512.2608
Epoch 323/5000
10/10 [==============================] - 0s 10ms/sample - loss: 0.0253 - mean_absolute_error: 0.0253 - mean_absolute_percentage_error: 8.2125 - val_loss: 1.8610 - val_mean_absolute_error: 1.8610 - val_mean_absolute_percentage_error: 502.8025
Epoch 324/5000
10/10 [==============================] - 0s 9ms/sample - loss: 0.0399 - mean_absolute_error: 0.0399 - mean_absolute_percentage_error: 12.4176 - val_loss: 1.8534 - val_mean_absolute_error: 1.8534 - val_mean_absolut

In [17]:
#save as excel
#===============

# Define column names for the Excel file
columns = ['Counter', 'lr', 'bs', 'MAE', 'val MAE', 'MAPE', 'val MAPE']
# Convert Results array to a DataFrame
Results_df = pd.DataFrame(Results, columns=columns)
writer = pd.ExcelWriter(path_data_2 + "Results_file_" + str(number_of_test) + ".xlsx", engine='xlsxwriter')
Results_df.to_excel(writer, sheet_name='Results_file_', index=False)
writer.save()

In [18]:
Results_df.head(20)

,Counter,lr,bs,MAE,val MAE,MAPE,val MAPE
0,0,0.0100,1000,0.137764,0.316515,40.670380,73.789017
1,1,0.0010,1000,0.046930,1.496931,12.250326,361.519196
2,2,0.0001,1000,0.018512,1.725523,4.805244,401.404785
3,3,0.0100,30,0.028062,0.241074,7.948962,44.626701
4,4,0.0010,30,0.102374,2.214418,26.120850,670.450867
5,5,0.0001,30,0.111882,3.055309,25.833797,981.264282
6,6,0.0100,15,0.053587,0.197184,17.897900,39.489357
7,7,0.0010,15,0.147886,3.689154,36.455952,1045.255127
8,8,0.0001,15,0.031834,3.200901,7.810313,996.301941
9,9,0.0100,5,0.124030,0.443540,35.121864,129.731049
